# Build a semantic search engine


In this model build with LangChain's document loader, embedding, and vector store abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation, or RAG (see our RAG tutorial here).

Here we will build a search engine over a PDF document. This will allow us to retrieve passages in the PDF that are similar to an input query.

### Concepts
This guide focuses on retrieval of text data. We will cover the following concepts:

Documents and document loaders;
Text splitters;
Embeddings;
Vector stores and retrievers.

## Setup

### Installation

In [1]:
!pip install langchain-community pypdf

In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


### Documents and Document Loaders
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has three attributes:

page_content: a string representing the content;\
metadata: a dict containing arbitrary metadata;\
id: (optional) a string identifier for the document.\
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

We can generate sample documents when desired:

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

However, the LangChain ecosystem implements document loaders that integrate with hundreds of common sources. This makes it easy to incorporate data from these sources into your AI application.

### Loading documents

Let's load a PDF into a sequence of Document objects. There is a sample PDF in the LangChain repo here -- a 10-k filing for Nike from 2023. We can consult the LangChain documentation for available PDF document loaders. Let's select PyPDFLoader, which is fairly lightweight.

In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "D:\Portfolio\Project\Langchain\semantic search engine/Nike-NPS-Combo_Form-10-K_WR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

106


PyPDFLoader loads one Document object per PDF page. For each, we can easily access:

The string content of the page;
Metadata containing the file name and page number.

In [5]:
print(f"{docs[10].page_content[:500]}\n")
print(docs[10].metadata)

Our DE&I focus extends beyond our workforce and includes our communities, which we support in a number of ways. We have 
committed to investments that aim to address racial inequality and improve diversity and representation in our communities. We 
also are leveraging our global scale to accelerate business diversity, including investing in business training programs for women 
and increasing the proportion of services supplied by minority-owned businesses.
COMPENSATION AND BENEFITS 
NIKE's tota

{'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'creator': 'Workiva', 'creationdate': '2023-07-20T22:09:22+00:00', 'author': 'anonymous', 'moddate': '2023-07-26T15:13:52+08:00', 'title': 'Nike 2023 Proxy', 'source': 'D:\\Portfolio\\Project\\Langchain\\semantic search engine/Nike-NPS-Combo_Form-10-K_WR.pdf', 'total_pages': 106, 'page': 10, 'page_label': '11'}


### Splitting
For both information retrieval and downstream question-answering purposes, a page may be too coarse a representation. Our goal in the end will be to retrieve Document objects that answer an input query, and further splitting our PDF will help ensure that the meanings of relevant portions of the document are not "washed out" by surrounding text.

We can use text splitters for this purpose. Here we will use a simple text splitter that partitions based on characters. We will split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it. We use the RecursiveCharacterTextSplitter, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We set add_start_index=True so that the character index where each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

See this guide for more detail about working with PDFs, including how to extract text from specific sections and images.

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

501

### Embeddings
Vector search is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can embed it as a vector of the same dimension and use vector similarity metrics (such as cosine similarity) to identify related text.

LangChain supports embeddings from dozens of providers. These models specify how text should be converted into a numeric vector. Let's select a model:

In [22]:
%pip install langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [25]:
!pip install sentence-transformers

  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.0 MB 8.4 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/12.0 MB 8.8 MB/s eta 0:00:01
   -------------------- ------------------- 6.0/12.0 MB 10.2 MB/s eta 0:00:01
   ----------------------------- ---------- 8.9/12.0 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
    --------------------------------------- 3.9/241.4 MB 18.1 MB/s eta 0:00:14
   - -------------------------------------- 8.7/241.4 MB 20.7 MB/s eta 0:00:12
   -- ------------------------------------- 14.2/241.4 MB 22.2 MB/s eta 0:00:11
   --- ------------------------------------ 20.4/241.4 MB 24.4 MB/s eta 0:00:10
   ---- ----

In [26]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\prath\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\prath\miniconda3\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prath\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develo

In [27]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.00521988095715642, -0.04748304933309555, -0.005584154278039932, -0.01912841945886612, -0.02318836748600006, -0.02301083318889141, -0.0322050116956234, 0.005130973644554615, 0.015710288658738136, -0.026600752025842667]


### Vector stores
LangChain VectorStore objects contain methods for adding text and Document objects to the store, and querying them using various similarity metrics. They are often initialized with embedding models, which determine how text data is translated to numeric vectors.

LangChain includes a suite of integrations with different vector store technologies. Some vector stores are hosted by a provider (e.g., various cloud providers) and require specific credentials to use; some (such as Postgres) run in separate infrastructure that can be run locally or via a third-party; others can run in-memory for lightweight workloads. Let's select a vector store:

In [28]:
pip install langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [30]:
ids = vector_store.add_documents(documents=all_splits)

Note that most vector store implementations will allow you to connect to an existing vector store-- e.g., by providing a client, index name, or other information. See the documentation for a specific integration for more detail.

Once we've instantiated a VectorStore that contains documents, we can query it. VectorStore includes methods for querying:

Synchronously and asynchronously;
By string query and by vector;
With and without returning similarity scores;
By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).
The methods will generally include a list of Document objects in their outputs.

### Usage
Embeddings typically represent text as a "dense" vector such that texts with similar meanings are geometrically close. This lets us retrieve relevant information just by passing in a question, without knowledge of any specific key-terms used in the document.

Return documents based on similarity to a string query:

In [31]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='our Greater China geography, occupied by employees focused on implementing our wholesale, NIKE Direct and merchandising 
strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of 
which are owned and three of which are leased. Two other distribution centers, one located in Indianapolis, Indiana and one 
located in Dayton, Tennessee, are leased and operated by third-party logistics providers. One distribution center for Converse is 
located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States, some of 
which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United 
States are located in Laakdal, Belgium; Taicang, China; Tomisato, Japan and Icheon, Korea, all of which we own.' metadata={'source': 'D:\\Portfolio\\Project\\Langchain\\sem

In [32]:
# Async query:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this 
"Annual Report"), the terms "we," "us," "our," "NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries 
and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, 
equipment, accessories and services. NIKE is the largest seller of athletic footwear and apparel in the world. We sell our products 
through NIKE Direct operations, which are comprised of both NIKE-owned retail stores and sales through our digital platforms 
(also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales 
representatives in nearly all countries around the world. We also offer interactive consumer services and experiences t

### Return scores:

In [33]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.3697006404399872

page_content='YEAR ENDED MAY 31,
(Dollars in millions) 2023 2022 2021
REVENUES
North America $ 21,608 $ 18,353 $ 17,179 
Europe, Middle East & Africa  13,418  12,479  11,456 
Greater China  7,248  7,547  8,290 
Asia Pacific & Latin America  6,431  5,955  5,343 
Global Brand Divisions  58  102  25 
Total NIKE Brand  48,763  44,436  42,293 
Converse  2,427  2,346  2,205 
Corporate  27  (72)  40 
TOTAL NIKE, INC. REVENUES $ 51,217 $ 46,710 $ 44,538 
EARNINGS BEFORE INTEREST AND TAXES
North America $ 5,454 $ 5,114 $ 5,089 
Europe, Middle East & Africa  3,531  3,293  2,435 
Greater China  2,283  2,365  3,243 
Asia Pacific & Latin America  1,932  1,896  1,530 
Global Brand Divisions  (4,841)  (4,262)  (3,656) 
Converse  676  669  543 
Corporate  (2,840)  (2,219)  (2,261) 
Interest expense (income), net  (6)  205  262 
TOTAL NIKE, INC. INCOME BEFORE INCOME TAXES $ 6,201 $ 6,651 $ 6,661 
ADDITIONS TO PROPERTY, PLANT AND EQUIPMENT
North America $ 283 $ 146 $ 98 
Europ

In [34]:
# Return documents based on similarity to an embedded query:

embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='to-end technology foundation, which we believe will further accelerate our digital transformation. We believe this unified approach 
will accelerate growth and unlock more efficiency for our business, while driving speed and responsiveness as we serve 
consumers globally.
FINANCIAL HIGHLIGHTS 
• In fiscal 2023, NIKE, Inc. achieved record Revenues of $51.2 billion, which increased 10% and 16% on a reported and 
currency-neutral basis, respectively 
• NIKE Direct revenues grew 14% from $18.7 billion in fiscal 2022 to $21.3 billion in fiscal 2023, and represented 
approximately 44% of total NIKE Brand revenues for fiscal 2023
• Gross margin for the fiscal year decreased 250 basis points to 43.5% primarily driven by higher product costs, higher 
markdowns and unfavorable changes in foreign currency exchange rates, partially offset by strategic pricing actions
• Inventories as of May 31, 2023 were $8.5 billion, flat compared to the prior year, driven by the actions we took thr

## Retrievers

LangChain VectorStore objects do not subclass Runnable. LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations). Although we can construct retrievers from vector stores, retrievers can interface with non-vector store sources of data, as well (such as external APIs).

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [35]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='2c6d5bc7-f680-4aca-b045-ec965207120d', metadata={'author': 'anonymous', 'source': 'D:\\Portfolio\\Project\\Langchain\\semantic search engine/Nike-NPS-Combo_Form-10-K_WR.pdf', 'moddate': '2023-07-26T15:13:52+08:00', 'title': 'Nike 2023 Proxy', 'total_pages': 106, 'start_index': 877, 'creationdate': '2023-07-20T22:09:22+00:00', 'page_label': '28', 'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'creator': 'Workiva', 'page': 27}, page_content='our Greater China geography, occupied by employees focused on implementing our wholesale, NIKE Direct and merchandising \nstrategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of \nwhich are owned and three of which are leased. Two other distribution centers, one located in Indianapolis, Indiana and one \nlocated in Dayton, Tennessee, are leased and operated by third-party logistics providers. O

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [36]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='2c6d5bc7-f680-4aca-b045-ec965207120d', metadata={'creator': 'Workiva', 'page': 27, 'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'total_pages': 106, 'page_label': '28', 'author': 'anonymous', 'moddate': '2023-07-26T15:13:52+08:00', 'start_index': 877, 'source': 'D:\\Portfolio\\Project\\Langchain\\semantic search engine/Nike-NPS-Combo_Form-10-K_WR.pdf', 'title': 'Nike 2023 Proxy', 'creationdate': '2023-07-20T22:09:22+00:00'}, page_content='our Greater China geography, occupied by employees focused on implementing our wholesale, NIKE Direct and merchandising \nstrategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of \nwhich are owned and three of which are leased. Two other distribution centers, one located in Indianapolis, Indiana and one \nlocated in Dayton, Tennessee, are leased and operated by third-party logistics providers. O


VectorStoreRetriever supports search types of "similarity" (default), "mmr" (maximum marginal relevance, described above), and "similarity_score_threshold". We can use the latter to threshold documents output by the retriever by similarity score.

Retrievers can easily be incorporated into more complex applications, such as retrieval-augmented generation (RAG) applications that combine a given question with retrieved context into a prompt for a LLM. To learn more about building such an application, check out the RAG tutorial tutorial.